In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib
import string

from os import path
from wordcloud import WordCloud

from nltk.stem import WordNetLemmatizer
import nltk.sentiment

%matplotlib inline

# Part 0: Loading and cleaning the data

**First we need to load the emails into a dataframe.**

In [ ]:
# Read Emails.csv into pandas
emails_dir = "hillary-clinton-emails"
emails_csv_filename = "Emails.csv"

emails_path = path.join(emails_dir, emails_csv_filename)

emails_df = pd.read_csv(emails_path)
len(emails_df)

In [ ]:
# Take a peek
emails_df.sample(5)

**We want to create our wordcloud from the column(s) that contain(s) the full text of the emails. From the schema, we know that the email subject should be in ExtractedSubject and email text should be in ExtractedBodyText. But we should make sure these fields are filled in!**

In [ ]:
# Which columns of the dataframe contain the text we want for our wordcloud?
#I found:
#- ExtractedSubject
#- ExtractedBodyText
#- RawText

# check these columns to see if they're actually filled in and make sense
emails_df.isnull().sum()

**Most rows have non-null ExtractedSubject and ExtractedBodyText, so we decided arbitrarily to drop all the emails that are missing these fields.**

In [ ]:
# In-place!
emails_df.dropna(subset=['ExtractedSubject', 'ExtractedBodyText'], inplace=True)
len(emails_df)

# Part 1: Generating wordclouds!

**We'll generate a wordcloud using the concatenated text of the entire email corpus.**

In [ ]:
#Concatenate all raw text
concatenated_body_text = ' '.join(emails_df['ExtractedSubject'] + ' ' + emails_df['ExtractedBodyText'])
len(concatenated_body_text)

# Generate a word cloud image
wordcloud = WordCloud().generate(concatenated_body_text)

# Display the generated image:
# the matplotlib way:
import matplotlib.pyplot as plt
plt.imshow(wordcloud)
plt.axis("off")

**The wordcloud library helpfully removed common english stopwords, but we still have some boring terms like 'RE' and 'FW' in our cloud!**

**We can improve the results by doing some pre-processing:**
- tokenization
- stopword removal (including email terms)
- lemmatization
- stemming (maybe?)
- remove punctuation & single char tokens

In [ ]:
import nltk
from nltk.corpus import stopwords
#NOTE to team you should run nltk.download() but maybe outside the notebook

In [ ]:
#Takes a few seconds to run:
tokenized = nltk.word_tokenize(concatenated_body_text)

In [ ]:
tokenized[:10]

**Let's remove some more stopwords**

In [ ]:
# Define annoying email terms as stopwords
email_stopwords = nltk.corpus.stopwords.words('english')
additional_stopwords = ['subject', 'date', 're', 'cc', 'bcc', 'fwd', 'fw', 'sent', 'mr', 'mrs']
email_stopwords.extend(additional_stopwords)

In [ ]:
tokens_without_stopwords = [t.lower() for t in tokenized if t.lower() not in email_stopwords]

**Lemmatization - we use the WordnetLemmatizer to group inflected forms of a word**

In [ ]:
lemmatizer = WordNetLemmatizer()
lemmatized_tokens = [lemmatizer.lemmatize(t) for t in tokens_without_stopwords]

**Stemming - we can use the porter stemmer to normalize our text**

In [ ]:
porter = nltk.PorterStemmer()
processed_tokens = [porter.stem(t) for t in lemmatized_tokens]

**Remove punctuation & single character token**

In [ ]:
def remove_punctuation(token):
    for c in string.punctuation:
        token = token.replace(c, "")
    
    return token

In [ ]:
processed_tokens = [token for token in processed_tokens if len(token) > 1]
processed_tokens = [remove_punctuation(token) for token in processed_tokens]

In [ ]:
processed_tokens[:10]

**Our NLTK pipeline (fully defined below) consists of tokenization, lower-case conversion, lemmatization and punctuation/single char removal.**

In [ ]:
# All processing at once. We don't apply stemming on emails, since it would destroy most country names
def process_text(text):
    tokens = nltk.word_tokenize(text)
    lowered = (t.lower() for t in tokens)
    processed = (lemmatizer.lemmatize(t) for t in lowered if t not in email_stopwords)
    processed = (remove_punctuation(t) for t in processed)
    processed = (t for t in processed if len(t) > 1)
            
    return list(processed)

In [ ]:
wc2_processed_tokens = process_text(concatenated_body_text)
wordcloud2 = WordCloud().generate(" ".join(wc2_processed_tokens))

plt.imshow(wordcloud2)
plt.axis("off")

**The second wordcloud looks a bit better thanks to the additional processing. Some conclusions:**
- (+) we were able to remove some of the mundane email terms from the text
- (~) lemmatization seemed more useful than stemming for generating a readable wordcloud.
- (-) removing words is sort of an iterative process as each time we generate the wordcloud we see new frequent words that we'd prefer to exclude!

# 2. World country mentions

**We use pycountry to find references in the emails to various countries**

In [ ]:
import pycountry

In [ ]:
countries = {country: 0 for country in pycountry.countries}

**First we go through all of the tokens and look for any mentions of countries**

In [ ]:
bad_country_tokens = set()
concatenated_body_text = " ".join(str(email) for email in emails_df["ExtractedBodyText"])

for token in process_text(concatenated_body_text):
    if token not in bad_country_tokens:
        try:
            country = pycountry.countries.lookup(token)
            countries[country] += 1
        except LookupError:
            bad_country_tokens.add(token)

In [ ]:
country_df = pd.DataFrame.from_dict({country.name: occurence for (country, occurence) in countries.items()}, orient="index")
country_df.columns = ["Occurences"]

In [ ]:
country_df.sort_values(by="Occurences", ascending=False).head(10)

In [ ]:
print(pycountry.countries.lookup("PM").name)
print(pycountry.countries.lookup("AM").name)

**Saint Pierre and Miquelon refers to "PM". "AM" was already in the stop-words, that's why we don't have problems with Armenia. We could start again with "pm" as a stopword, but Saint Pierre and Miquelon is very likely not to appear in any email.**

In [ ]:
country_df.loc["Saint Pierre and Miquelon"] = 0
country_df.sort_values(by="Occurences", ascending=False).head(10)

In [ ]:
vader = nltk.sentiment.vader.SentimentIntensityAnalyzer()

In [ ]:
vader.polarity_scores("I love you")

In [ ]:
vader.polarity_scores("I hate you")

In [ ]:
email_stopwords.append("pm")

In [ ]:
import os.path
import pickle

# Create cache folder
CACHE_DIR = "cache"
CACHE_PATH = os.path.join(CACHE_DIR, "country_sentiment_df.bak")

if not os.path.exists(CACHE_DIR):
    os.makedirs(CACHE_DIR)

In [ ]:
country_sentiment_list = []

# !! If not in the cache, takes ~10 min to run
if os.path.isfile(CACHE_PATH):
    with open(CACHE_PATH, 'rb') as handle:
        country_sentiment_df = pickle.load(handle)

else:
    for (index, email) in emails_df.iterrows():
        text = email["ExtractedBodyText"]
        words = process_text(str(text))

        for word in words:
            if word not in bad_country_tokens:
                country = pycountry.countries.lookup(word)
                sentiment = vader.polarity_scores(text)["compound"]
                country_sentiment_list.append([country.name, sentiment])
                break

    country_sentiment_df = pd.DataFrame(country_sentiment_list, columns=["Country", "Sentiment"])
    
    with open(CACHE_PATH, 'wb') as handle:
        pickle.dump(country_sentiment_df, handle)

In [ ]:
sentiment_by_country = country_sentiment_df.groupby("Country")


sentiment_occurence_df = sentiment_by_country.count().join(sentiment_by_country.mean(), lsuffix='_left')
sentiment_occurence_df.columns = ["Frequency", "Sentiment"]

**First, a full & difficult to read plot of sentiment towards countries. Then, we only show countries with most positive / most negative sentiment. We can see that the emails globally have a positive sentiment.**

In [ ]:
palette = sns.color_palette("coolwarm_r", n_colors=len(sentiment_occurence_df))
matplotlib.rcParams['figure.figsize'] = (10.0, 8.0)
sentiment_occurence_df["Sentiment"].sort_values().plot(kind="bar", color=palette)

**Below is a plot of the countries towards which Hillary & co. expressed the most negative sentiment**

In [ ]:
matplotlib.rcParams['figure.figsize'] = (8, 4)

# Most negative sentiment
sentiment_occurence_df["Sentiment"].sort_values().head(25).plot(kind="bar", color=palette)

**Similarly, here is a plot of the countries towards which Hillary & co. expressed the most positive sentiment**

In [ ]:
# Most positive sentiment
sentiment_occurence_df["Sentiment"].sort_values().tail(25).plot(kind="bar")

**NOTE: There are some surprising countries at the top of the sentiment list!**

# 3. Topic modeling

In [ ]:
import gensim.models.ldamodel as ldamodel
import gensim.corpora as corpora

**For this task we use the full emails (including subject), process the text with our NLTK piepline, then create LDA modes with varying numbers of topics**

In [ ]:
texts = [process_text(str(email)) for email in emails_df['ExtractedSubject'] + " " + emails_df["ExtractedBodyText"]]
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [ ]:
lda = ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=10)

In [ ]:
def pretty_print_topics(lda):
    for topics in lda.show_topics(num_topics=20, num_words=8, formatted=False):
        print([topic[0] for topic in topics[1]])
        

In [ ]:
# !! Takes a few minutes
models = dict()
for topics_count in range(5, 50, 10):
    print("{} topics".format(topics_count))
    models[topics_count] = ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=topics_count)
    pretty_print_topics(models[topics_count])
    print()

**It's very difficult to tell without more knowledge of the email contents, but it looks like 45 topics gives the most meaningful results. There are some pretty stable topics across the different iterations, for example Israeli-Arab-Palestinian relations.**